In [1]:
import sys, os
import numpy as np
import pandas as pd
import random
from random import shuffle, choice
import time
import os
import glob
import keras
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from random import shuffle, choice
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
from keras.models import Model
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler

batch_size = 250
epochs = 100
epochs_traits = 500
num_classes = 3

#From: https://towardsdatascience.com/neural-networks-ensemble-33f33bea7df3
class LinearW(Layer):

    def __init__(self):
        super(LinearW, self).__init__()
        
    def build(self, input_shape):

        self.W = self.add_weight(name='name',
                    shape=(1,1,len(input_shape)),
                    initializer='uniform',
                    dtype=tf.float32,
                    trainable=True)
        
    def call(self, inputs):

        # inputs is a list of tensor of shape [(n_batch, n_feat), ..., (n_batch, n_feat)]
        # expand last dim of each input passed [(n_batch, n_feat, 1), ..., (n_batch, n_feat, 1)]
        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs) # (n_batch, n_feat, n_inputs)
        weights = tf.nn.softmax(self.W, axis=-1) # (1,1,n_inputs)
        # weights sum up to one on last dim

        return tf.reduce_sum(weights*inputs, axis=-1) # (n_batch, n_feat)

def create_mlp(traitstrain, regularizer=None):
  """Creates a three-layer MLP with inputs of the given dimension"""
  model = Sequential()
  model.add(Dense(150, use_bias=False, input_dim=traitstrain.shape[1], activation="relu", kernel_regularizer=regularizers.l1(0.001)))
  model.add(BatchNormalization())
  #model.add(Dropout(0.5))
  model.add(Dense(150, use_bias=False, activation="relu"))
  model.add(BatchNormalization())
  #model.add(Dropout(0.5))
  model.add(Dense(50, activation="relu"))
  return model
  
def create_cnn(xtest, regularizer=None):
  inputShape = (xtest.shape[1], xtest.shape[2])
  inputs = Input(shape=inputShape)
  x = inputs
  x = Conv1D(250, kernel_size=3, activation='relu', use_bias=False, input_shape=(xtest.shape[1], xtest.shape[2]))(x)
  x = BatchNormalization()(x)
  x = Conv1D(250, kernel_size=3, use_bias=False, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Conv1D(250, kernel_size=3, use_bias=False, activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPooling1D(pool_size=3)(x)
  x = Flatten()(x)
  x = Dense(125, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(125, activation='relu')(x)
  x = Dropout(0.5)(x)
  # Apply another fully-connected layer, this one to match the number of nodes coming out of the MLP
  x = Dense(50, kernel_regularizer=regularizer)(x)
  x = Activation("relu")(x)
  # Construct the CNN
  model = Model(inputs, x)
  # Return the CNN
  return model

2023-03-24 16:57:30.656471: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 16:57:30.773666: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
################################################################################################################################################
#Load data
################################################################################################################################################

traits_BM = []
traits_BM = np.loadtxt("./traits/traits_BM.txt").reshape(30000,-1,28)

scalers_BM = {}
for i in range(traits_BM.shape[2]):
    scalers_BM[i] = StandardScaler(copy=False)
    traits_BM[:, :, i] = scalers_BM[i].fit_transform(traits_BM[:, :, i]) 

traits_BM = np.array(traits_BM)

#Add missing individuals
for i in range(traits_BM.shape[0]):
  for j in range(0,4):
    traits_BM[i,j,:] = 0
  for j in range(77,81):
    traits_BM[i,j,:] = 0
  for j in range(134,136):
    traits_BM[i,j,:] = 0

d_traits = []
d_traits = np.loadtxt("./traits/traits_disc.txt").reshape(30000,-1,3)
traits = np.concatenate((traits_BM,d_traits),axis=2)

u1 = np.load("./trainingSims/Model_2sp.npz")
u2 = np.load("./trainingSims/Model_6sp.npz")
u3 = np.load("./trainingSims/Model_6spMig.npz")

x=np.concatenate((u1['Model_2sp'],u2['Model_6sp'],u3['Model_6spMig']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(x):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      x[arr][idx][x[arr][idx] == 1] = -1
      x[arr][idx][x[arr][idx] == 0] = 1
    else:
      x[arr][idx][x[arr][idx] == 0] = -1

y=[0 for i in range(len(u1['Model_2sp']))]
y.extend([1 for i in range(len(u2['Model_6sp']))])
y.extend([2 for i in range(len(u3['Model_6spMig']))])
y = np.array(y)

print (len(x), len(y), len(traits))
shf = list(range(len(x)))
shuffle(shf)

y = y[shf]
x = x[shf]
traits = traits[shf]


#Add missing data as 0s, according to a specifies missing data percentage
#54,477,268 SNP datapoints and 24,326,002 missing genotypes = 44.6%
missD_perc = 44.6
missD = int(x.shape[1]*x.shape[2]*(missD_perc/100))
for i in range(x.shape[0]):
  for m in range(missD):
    j = random.randint(0, x.shape[1] - 1)
    k = random.randint(0, x.shape[2] - 1)
    x[i][j][k] = 0

del(missD)

xtrain, xtest = x[int(len(y)*.25):], x[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
traits_train, traits_test = traits[int(len(y)*.25):], traits[:int(len(y)*.25)]


ytest = np_utils.to_categorical(ytest, num_classes)
ytrain = np_utils.to_categorical(ytrain, num_classes)
traits_train=traits_train.reshape((traits_train.shape[0], (traits_train.shape[1]*traits_train.shape[2])))
traits_test=traits_test.reshape((traits_test.shape[0], (traits_test.shape[1]*traits_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_train)
cnn = create_cnn(xtest)

30000 30000 30000


2023-03-24 20:12:09.374056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 20:12:09.992051: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22129 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:73:00.0, compute capability: 8.6


In [3]:
################################################################################################################################################
#Train the combined SNPs + traits (BM) network
################################################################################################################################################

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one sigmoid)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)

opt = SGD(learning_rate=0.001)

model1.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model1.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=100, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model1.fit([traits_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

model1.save(filepath='./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1000, 458)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 998, 250)     343500      ['input_1[0][0]']                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 998, 250)    1000        ['conv1d[0][0]']                 
 rmalization)                                                                                     
                                                                                                  
 conv1d_1 (Conv1D)              (None, 996, 250)     187500      ['batch_normalization_2[0][

2023-03-24 14:07:12.149169: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-24 14:07:12.388667: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-03-24 14:07:14.112088: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-24 14:07:14.113883: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-24 14:07:14.113901: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-24 14:07:14.115818: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-24 14:07:14.123042: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to

90/90 [==============================] - 31s 261ms/step - loss: 16.4565 - accuracy: 0.3603 - val_loss: 16.3121 - val_accuracy: 0.3931
Epoch 2/100
90/90 [==============================] - 16s 174ms/step - loss: 16.2275 - accuracy: 0.4519 - val_loss: 16.1194 - val_accuracy: 0.5404
Epoch 3/100
90/90 [==============================] - 17s 190ms/step - loss: 15.9983 - accuracy: 0.5470 - val_loss: 15.8367 - val_accuracy: 0.6585
Epoch 4/100
90/90 [==============================] - 15s 169ms/step - loss: 15.7828 - accuracy: 0.6038 - val_loss: 15.5951 - val_accuracy: 0.6929
Epoch 5/100
90/90 [==============================] - 16s 175ms/step - loss: 15.6031 - accuracy: 0.6386 - val_loss: 15.4226 - val_accuracy: 0.6956
Epoch 6/100
90/90 [==============================] - 18s 196ms/step - loss: 15.4579 - accuracy: 0.6545 - val_loss: 15.2857 - val_accuracy: 0.7085
Epoch 7/100
90/90 [==============================] - 15s 171ms/step - loss: 15.3244 - accuracy: 0.6712 - val_loss: 15.1646 - val_accurac

Epoch 57/100
90/90 [==============================] - 17s 184ms/step - loss: 10.5375 - accuracy: 0.9439 - val_loss: 10.5199 - val_accuracy: 0.9277
Epoch 58/100
90/90 [==============================] - 16s 179ms/step - loss: 10.4559 - accuracy: 0.9440 - val_loss: 10.4237 - val_accuracy: 0.9379
Epoch 59/100
90/90 [==============================] - 15s 168ms/step - loss: 10.3755 - accuracy: 0.9454 - val_loss: 10.3256 - val_accuracy: 0.9488
Epoch 60/100
90/90 [==============================] - 15s 169ms/step - loss: 10.2906 - accuracy: 0.9468 - val_loss: 10.2428 - val_accuracy: 0.9504
Epoch 61/100
90/90 [==============================] - 17s 189ms/step - loss: 10.2080 - accuracy: 0.9502 - val_loss: 10.1622 - val_accuracy: 0.9535
Epoch 62/100
90/90 [==============================] - 16s 178ms/step - loss: 10.1222 - accuracy: 0.9532 - val_loss: 10.0811 - val_accuracy: 0.9527
Epoch 63/100
90/90 [==============================] - 16s 182ms/step - loss: 10.0449 - accuracy: 0.9535 - val_loss: 10

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod/assets


INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod/assets


In [5]:
################################################################################################################################################
#Train the SNPs only network
################################################################################################################################################

xCNN = Dense(num_classes, activation="softmax")(cnn.output)

model2 = Model(inputs=cnn.input, outputs=xCNN)

opt = SGD(learning_rate=0.001)

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=100, verbose=0, mode='max', restore_best_weights=True)

model2.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model2.summary())

start = time.time()
model2.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

model2.save(filepath='./TrainedModels/Trained_CNN_Model_1KSNPs.acc.mod')


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000, 458)]       0         
                                                                 
 conv1d (Conv1D)             (None, 998, 250)          343500    
                                                                 
 batch_normalization_2 (Batc  (None, 998, 250)         1000      
 hNormalization)                                                 
                                                                 
 conv1d_1 (Conv1D)           (None, 996, 250)          187500    
                                                                 
 batch_normalization_3 (Batc  (None, 996, 250)         1000      
 hNormalization)                                                 
                                                                 
 conv1d_2 (Conv1D)           (None, 994, 250)          1875

2023-03-27 12:19:48.619536: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-03-27 12:19:50.120208: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-27 12:19:50.121849: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-27 12:19:50.121864: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-27 12:19:50.123632: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-27 12:19:50.123701: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-03-27 12:19:51.159244: I tensorflow/stream_executor/cuda/c

90/90 [==============================] - 27s 226ms/step - loss: 1.1498 - accuracy: 0.3916 - val_loss: 1.0245 - val_accuracy: 0.5915
Epoch 2/100
90/90 [==============================] - 17s 185ms/step - loss: 0.9620 - accuracy: 0.4923 - val_loss: 0.8730 - val_accuracy: 0.5973
Epoch 3/100
90/90 [==============================] - 15s 166ms/step - loss: 0.8557 - accuracy: 0.5618 - val_loss: 0.7225 - val_accuracy: 0.6416
Epoch 4/100
90/90 [==============================] - 16s 174ms/step - loss: 0.7763 - accuracy: 0.6035 - val_loss: 0.6010 - val_accuracy: 0.6971
Epoch 5/100
90/90 [==============================] - 16s 182ms/step - loss: 0.7222 - accuracy: 0.6250 - val_loss: 0.5476 - val_accuracy: 0.7041
Epoch 6/100
90/90 [==============================] - 15s 167ms/step - loss: 0.6761 - accuracy: 0.6516 - val_loss: 0.5179 - val_accuracy: 0.7083
Epoch 7/100
90/90 [==============================] - 17s 186ms/step - loss: 0.6490 - accuracy: 0.6620 - val_loss: 0.4974 - val_accuracy: 0.7313
Epoc

90/90 [==============================] - 15s 165ms/step - loss: 0.0948 - accuracy: 0.9673 - val_loss: 0.1381 - val_accuracy: 0.9481
Epoch 58/100
90/90 [==============================] - 16s 173ms/step - loss: 0.0923 - accuracy: 0.9674 - val_loss: 0.1400 - val_accuracy: 0.9483
Epoch 59/100
90/90 [==============================] - 16s 180ms/step - loss: 0.0863 - accuracy: 0.9713 - val_loss: 0.1391 - val_accuracy: 0.9499
Epoch 60/100
90/90 [==============================] - 16s 180ms/step - loss: 0.0853 - accuracy: 0.9706 - val_loss: 0.1425 - val_accuracy: 0.9501
Epoch 61/100
90/90 [==============================] - 15s 171ms/step - loss: 0.0794 - accuracy: 0.9727 - val_loss: 0.1407 - val_accuracy: 0.9513
Epoch 62/100
90/90 [==============================] - 15s 172ms/step - loss: 0.0808 - accuracy: 0.9724 - val_loss: 0.1363 - val_accuracy: 0.9497
Epoch 63/100
90/90 [==============================] - 15s 169ms/step - loss: 0.0773 - accuracy: 0.9735 - val_loss: 0.1351 - val_accuracy: 0.951

INFO:tensorflow:Assets written to: Trained_CNN_Model_1KSNPs.acc.mod/assets


INFO:tensorflow:Assets written to: Trained_CNN_Model_1KSNPs.acc.mod/assets


In [6]:
################################################################################################################################################
#Train the traits only (BM) network
################################################################################################################################################

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())

start = time.time()
model3.fit(traits_train, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(traits_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')
          
model3.save(filepath='./TrainedModels/Trained_Traits_Model_BM.acc.mod')

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_input (InputLayer)    [(None, 7099)]            0         
                                                                 
 dense (Dense)               (None, 150)               1064850   
                                                                 
 batch_normalization (BatchN  (None, 150)              600       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 150)               22500     
                                                                 
 batch_normalization_1 (Batc  (None, 150)              600       
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 50)                7550

90/90 [==============================] - 1s 10ms/step - loss: 11.5416 - accuracy: 0.8984 - val_loss: 11.7912 - val_accuracy: 0.6879
Epoch 48/100
90/90 [==============================] - 1s 10ms/step - loss: 11.4524 - accuracy: 0.9048 - val_loss: 11.7060 - val_accuracy: 0.6891
Epoch 49/100
90/90 [==============================] - 1s 10ms/step - loss: 11.3629 - accuracy: 0.9075 - val_loss: 11.6226 - val_accuracy: 0.6911
Epoch 50/100
90/90 [==============================] - 1s 10ms/step - loss: 11.2758 - accuracy: 0.9102 - val_loss: 11.5396 - val_accuracy: 0.6915
Epoch 51/100
90/90 [==============================] - 1s 10ms/step - loss: 11.1854 - accuracy: 0.9146 - val_loss: 11.4573 - val_accuracy: 0.6924
Epoch 52/100
90/90 [==============================] - 1s 10ms/step - loss: 11.0966 - accuracy: 0.9187 - val_loss: 11.3740 - val_accuracy: 0.6940
Epoch 53/100
90/90 [==============================] - 1s 10ms/step - loss: 11.0104 - accuracy: 0.9195 - val_loss: 11.2922 - val_accuracy: 0.695

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Traits_Model_BM.acc.mod/assets


In [43]:
################################################################################################################################################
#Load OU traits
################################################################################################################################################

traits_OU = []
traits_OU = np.loadtxt("./traits/trait x  s_OU.txt").reshape(30000,-1,28)

scalers_OU = {}
for i in range(traits_OU.shape[2]):
    scalers_OU[i] = StandardScaler(copy=False)
    traits_OU[:, :, i] = scalers_OU[i].fit_transform(traits_OU[:, :, i]) 

traits_OU = np.array(traits_OU)

#Add missing individuals
for i in range(traits_OU.shape[0]):
  for j in range(0,4):
    traits_OU[i,j,:] = 0
  for j in range(77,81):
    traits_OU[i,j,:] = 0
  for j in range(134,136):
    traits_OU[i,j,:] = 0

d_traits = []
d_traits = np.loadtxt("./traits/traits_disc.txt").reshape(30000,-1,3)
traits = np.concatenate((traits_OU,d_traits),axis=2)

u1 = np.load("./trainingSims/Model_2sp.npz")
u2 = np.load("./trainingSims/Model_6sp.npz")
u3 = np.load("./trainingSims/Model_6spMig.npz")

x=np.concatenate((u1['Model_2sp'],u2['Model_6sp'],u3['Model_6spMig']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(x):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      x[arr][idx][x[arr][idx] == 1] = -1
      x[arr][idx][x[arr][idx] == 0] = 1
    else:
      x[arr][idx][x[arr][idx] == 0] = -1

y=[0 for i in range(len(u1['Model_2sp']))]
y.extend([1 for i in range(len(u2['Model_6sp']))])
y.extend([2 for i in range(len(u3['Model_6spMig']))])
y = np.array(y)

print (len(x), len(y), len(traits))
shf = list(range(len(x)))
shuffle(shf)

y = y[shf]
x = x[shf]
traits = traits[shf]


#Add missing data as 0s, according to a specifies missing data percentage
#54,477,268 SNP datapoints and 24,326,002 missing genotypes = 44.6%
missD_perc = 44.6
missD = int(x.shape[1]*x.shape[2]*(missD_perc/100))
for i in range(x.shape[0]):
  for m in range(missD):
    j = random.randint(0, x.shape[1] - 1)
    k = random.randint(0, x.shape[2] - 1)
    x[i][j][k] = 0

del(missD)

xtrain, xtest = x[int(len(y)*.25):], x[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
traits_train, traits_test = traits[int(len(y)*.25):], traits[:int(len(y)*.25)]


ytest = np_utils.to_categorical(ytest, num_classes)
ytrain = np_utils.to_categorical(ytrain, num_classes)
traits_train=traits_train.reshape((traits_train.shape[0], (traits_train.shape[1]*traits_train.shape[2])))
traits_test=traits_test.reshape((traits_test.shape[0], (traits_test.shape[1]*traits_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_train)
cnn = create_cnn(xtest)

30000 30000 30000


In [44]:
################################################################################################################################################
#Train the combined SNPs + traits (OU) network
################################################################################################################################################

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one sigmoid)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)

opt = SGD(learning_rate=0.001)

model1.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model1.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=100, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model1.fit([traits_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

model1.save(filepath='./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod')

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1000, 458)]  0           []                               
                                                                                                  
 conv1d_6 (Conv1D)              (None, 998, 250)     343500      ['input_3[0][0]']                
                                                                                                  
 batch_normalization_14 (BatchN  (None, 998, 250)    1000        ['conv1d_6[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 conv1d_7 (Conv1D)              (None, 996, 250)     187500      ['batch_normalization_14[0]

90/90 [==============================] - 16s 175ms/step - loss: 14.3863 - accuracy: 0.7036 - val_loss: 14.2810 - val_accuracy: 0.7267
Epoch 17/100
90/90 [==============================] - 15s 169ms/step - loss: 14.2891 - accuracy: 0.7050 - val_loss: 14.1873 - val_accuracy: 0.7327
Epoch 18/100
90/90 [==============================] - 15s 166ms/step - loss: 14.1901 - accuracy: 0.7152 - val_loss: 14.0915 - val_accuracy: 0.7437
Epoch 19/100
90/90 [==============================] - 15s 168ms/step - loss: 14.0922 - accuracy: 0.7180 - val_loss: 13.9979 - val_accuracy: 0.7436
Epoch 20/100
90/90 [==============================] - 15s 162ms/step - loss: 13.9961 - accuracy: 0.7217 - val_loss: 13.9059 - val_accuracy: 0.7425
Epoch 21/100
90/90 [==============================] - 15s 163ms/step - loss: 13.9065 - accuracy: 0.7231 - val_loss: 13.8142 - val_accuracy: 0.7473
Epoch 22/100
90/90 [==============================] - 15s 163ms/step - loss: 13.8068 - accuracy: 0.7290 - val_loss: 13.7219 - val_a

Epoch 72/100
90/90 [==============================] - 16s 176ms/step - loss: 9.3678 - accuracy: 0.9531 - val_loss: 9.3439 - val_accuracy: 0.9452
Epoch 73/100
90/90 [==============================] - 16s 180ms/step - loss: 9.2930 - accuracy: 0.9544 - val_loss: 9.2630 - val_accuracy: 0.9488
Epoch 74/100
90/90 [==============================] - 15s 168ms/step - loss: 9.2143 - accuracy: 0.9555 - val_loss: 9.2015 - val_accuracy: 0.9436
Epoch 75/100
90/90 [==============================] - 15s 169ms/step - loss: 9.1367 - accuracy: 0.9580 - val_loss: 9.1064 - val_accuracy: 0.9535
Epoch 76/100
90/90 [==============================] - 15s 171ms/step - loss: 9.0542 - accuracy: 0.9606 - val_loss: 9.0365 - val_accuracy: 0.9513
Epoch 77/100
90/90 [==============================] - 15s 170ms/step - loss: 8.9793 - accuracy: 0.9609 - val_loss: 8.9648 - val_accuracy: 0.9509
Epoch 78/100
90/90 [==============================] - 15s 168ms/step - loss: 8.9057 - accuracy: 0.9619 - val_loss: 8.8879 - val_ac

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod/assets


INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod/assets


In [45]:
################################################################################################################################################
#Train the traits only (OU) network
################################################################################################################################################

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())

start = time.time()
model3.fit(traits_train, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(traits_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')
          
model3.save(filepath='./TrainedModels/Trained_Traits_Model_OU.acc.mod')

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22_input (InputLayer)  [(None, 7099)]           0         
                                                                 
 dense_22 (Dense)            (None, 150)               1064850   
                                                                 
 batch_normalization_12 (Bat  (None, 150)              600       
 chNormalization)                                                
                                                                 
 dense_23 (Dense)            (None, 150)               22500     
                                                                 
 batch_normalization_13 (Bat  (None, 150)              600       
 chNormalization)                                                
                                                                 
 dense_24 (Dense)            (None, 50)                755

Epoch 48/100
90/90 [==============================] - 1s 11ms/step - loss: 4.5547 - accuracy: 0.9956 - val_loss: 5.1244 - val_accuracy: 0.7000
Epoch 49/100
90/90 [==============================] - 1s 11ms/step - loss: 4.4974 - accuracy: 0.9963 - val_loss: 5.0700 - val_accuracy: 0.6988
Epoch 50/100
90/90 [==============================] - 1s 11ms/step - loss: 4.4410 - accuracy: 0.9971 - val_loss: 5.0231 - val_accuracy: 0.7001
Epoch 51/100
90/90 [==============================] - 1s 11ms/step - loss: 4.3852 - accuracy: 0.9975 - val_loss: 4.9723 - val_accuracy: 0.7001
Epoch 52/100
90/90 [==============================] - 1s 11ms/step - loss: 4.3314 - accuracy: 0.9977 - val_loss: 4.9195 - val_accuracy: 0.6988
Epoch 53/100
90/90 [==============================] - 1s 11ms/step - loss: 4.2758 - accuracy: 0.9983 - val_loss: 4.8726 - val_accuracy: 0.7003
Epoch 54/100
90/90 [==============================] - 1s 13ms/step - loss: 4.2214 - accuracy: 0.9984 - val_loss: 4.8206 - val_accuracy: 0.7007

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Traits_Model_OU.acc.mod/assets


In [46]:
################################################################################################################################################
#Perform cross-validation
################################################################################################################################################


model1 = load_model('./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod')
model2 = load_model('./TrainedModels/Trained_CNN_Model_1KSNPs.acc.mod')
model3 = load_model('./TrainedModels/Trained_Traits_Model_BM.acc.mod')
model4 = load_model('./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod')
model5 = load_model('./TrainedModels/Trained_Traits_Model_OU.acc.mod')

traits_BM = []
traits_BM = np.loadtxt("./testSims/traits/traits_BM.txt").reshape(3000,-1,28)

for i in range(traits_BM.shape[2]):
    traits_BM[:, :, i] = scalers_BM[i].transform(traits_BM[:, :, i]) 

traits_BM = np.array(traits_BM)

#Add missing individuals
for i in range(traits_BM.shape[0]):
  for j in range(0,4):
    traits_BM[i,j,:] = 0
  for j in range(77,81):
    traits_BM[i,j,:] = 0
  for j in range(134,136):
    traits_BM[i,j,:] = 0

d_traits = []
d_traits = np.loadtxt("./testSims/traits/traits_disc.txt").reshape(3000,-1,3)
traits = np.concatenate((traits_BM,d_traits),axis=2)

u1 = np.load("./testSims/Model_2sp.npz")
u2 = np.load("./testSims/Model_6sp.npz")
u3 = np.load("./testSims/Model_6spMig.npz")

xtest=np.concatenate((u1['Model_2sp'],u2['Model_6sp'],u3['Model_6spMig']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(xtest):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      xtest[arr][idx][xtest[arr][idx] == 1] = -1
      xtest[arr][idx][xtest[arr][idx] == 0] = 1
    else:
      xtest[arr][idx][xtest[arr][idx] == 0] = -1

#Add missing data as 0s, according to a specifies missing data percentage
#54,477,268 SNP datapoints and 24,326,002 missing genotypes = 44.6%
missD_perc = 44.6
missD = int(xtest.shape[1]*xtest.shape[2]*(missD_perc/100))
for i in range(xtest.shape[0]):
  for m in range(missD):
    j = random.randint(0, xtest.shape[1] - 1)
    k = random.randint(0, xtest.shape[2] - 1)
    xtest[i][j][k] = 0


ytest=[0 for i in range(len(u1['Model_2sp']))]
ytest.extend([1 for i in range(len(u2['Model_6sp']))])
ytest.extend([2 for i in range(len(u3['Model_6spMig']))])
ytest = np.array(ytest)

#first get the predictions
pred = model1.predict([traits.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))


#first get the predictions
pred = model2.predict(xtest)
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

#first get the predictions
pred = model3.predict(traits.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

traits_OU = []
traits_OU = np.loadtxt("./testSims/traits/traits_OU.txt").reshape(3000,-1,28)

for i in range(traits_OU.shape[2]):
    traits_OU[:, :, i] = scalers_OU[i].transform(traits_OU[:, :, i]) 

traits_OU = np.array(traits_OU)

#Add missing individuals
for i in range(traits_OU.shape[0]):
  for j in range(0,4):
    traits_OU[i,j,:] = 0
  for j in range(77,81):
    traits_OU[i,j,:] = 0
  for j in range(134,136):
    traits_OU[i,j,:] = 0

d_traits = []
d_traits = np.loadtxt("./testSims/traits/traits_disc.txt").reshape(3000,-1,3)
traits = np.concatenate((traits_OU,d_traits),axis=2)

#first get the predictions
pred = model4.predict([traits.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))


#first get the predictions
pred = model5.predict(traits.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

94/94 [==============================] - 1s 11ms/step
[[999   0   1]
 [  0 958  42]
 [  0  79 921]]
94/94 [==============================] - 1s 10ms/step
[[998   0   2]
 [  0 941  59]
 [  0  66 934]]
94/94 [==============================] - 0s 3ms/step
[[940  14  46]
 [ 35 686 279]
 [ 80 361 559]]
94/94 [==============================] - 1s 11ms/step
[[999   0   1]
 [  0 986  14]
 [  0 110 890]]
94/94 [==============================] - 0s 3ms/step
[[944  18  38]
 [ 52 653 295]
 [107 389 504]]


In [47]:
################################################################################################################################################
#Predict empirical data
################################################################################################################################################


inSNPs=np.loadtxt("./input_T.txt")
inp=np.array(inSNPs)
num_samples=100
res = []
for i in range(0,num_samples):
	idx = np.random.choice(inp.shape[0], 1000, replace=False)
	n = inp[idx,:]
	res.append(np.array(n))

EmpSNPs = np.array(res)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(EmpSNPs):
  for idx,row in enumerate(array):
    if np.count_nonzero(row==1) > np.count_nonzero(row==-1):
      EmpSNPs[arr][idx][EmpSNPs[arr][idx] == 1] = 9
      EmpSNPs[arr][idx][EmpSNPs[arr][idx] == -1] = 1
      EmpSNPs[arr][idx][EmpSNPs[arr][idx] == 9] = -1

Laqu_Lpel=np.genfromtxt("./input_traits_Laqu_Lpel.txt", delimiter="\t", filling_values=0)
Laqu=np.genfromtxt("./input_traits_Laqu.txt", delimiter="\t", filling_values=0)
Lmeg=np.genfromtxt("./input_traits_Lmeg.txt", delimiter="\t", filling_values=0)
Loua=np.genfromtxt("./input_traits_Loua.txt", delimiter="\t", filling_values=0)
Lozk_Lmeg=np.genfromtxt("./input_traits_Lozk_Lmeg.txt", delimiter="\t", filling_values=0)
Lozk=np.genfromtxt("./input_traits_Lozk.txt", delimiter="\t", filling_values=0)
Lpel_Lmeg=np.genfromtxt("./input_traits_Lpel_Lmeg.txt", delimiter="\t", filling_values=0)
Lpel=np.genfromtxt("./input_traits_Lpel.txt", delimiter="\t", filling_values=0)
Lsol_Lmeg=np.genfromtxt("./input_traits_Lsol_Lmeg.txt", delimiter="\t", filling_values=0)
Lsol=np.genfromtxt("./input_traits_Lsol.txt", delimiter="\t", filling_values=0)

Laqu_Lpel=np.array(Laqu_Lpel)
Laqu=np.array(Laqu)
Lmeg=np.array(Lmeg)
Loua=np.array(Loua)
Lozk_Lmeg=np.array(Lozk_Lmeg)
Lozk=np.array(Lozk)
Lpel_Lmeg=np.array(Lpel_Lmeg)
Lpel=np.array(Lpel)
Lsol_Lmeg=np.array(Lsol_Lmeg)
Lsol=np.array(Lsol)


res = []
for i in range(0,num_samples):
  n = np.zeros((4,31))
  idx_aqu_pel = np.random.choice(Laqu_Lpel.shape[0], 6, replace=False)
  n = np.concatenate((n,Laqu_Lpel[idx_aqu_pel,:]), axis=0)
  idx_aqu = np.random.choice(Laqu.shape[0], 68, replace=False)
  n = np.concatenate((n,Laqu[idx_aqu,:]), axis=0)
  n = np.concatenate((n,np.zeros((3,31))), axis=0)
  idx_meg = np.random.choice(Lmeg.shape[0], 44, replace=False)
  n = np.concatenate((n,Lmeg[idx_meg,:]), axis=0)
  idx_oua = np.random.choice(Loua.shape[0], 10, replace=False)
  n = np.concatenate((n,Loua[idx_oua,:]), axis=0)
  n = np.concatenate((n,np.zeros((1,31))), axis=0)
  idx_ozk_meg = np.random.choice(Lozk_Lmeg.shape[0], 8, replace=False)
  n = np.concatenate((n,Lozk_Lmeg[idx_ozk_meg,:]), axis=0)
  idx_ozk = np.random.choice(Lozk.shape[0], 15, replace=False)
  n = np.concatenate((n,Lozk[idx_ozk,:]), axis=0)
  idx_pel_meg = np.random.choice(Lpel_Lmeg.shape[0], 10, replace=False)
  n = np.concatenate((n,Lpel_Lmeg[idx_pel_meg,:]), axis=0)
  idx_pel = np.random.choice(Lpel.shape[0], 19, replace=False)
  n = np.concatenate((n,Lpel[idx_pel,:]), axis=0)
  idx_sol_meg = np.random.choice(Lsol_Lmeg.shape[0], 5, replace=False)
  n = np.concatenate((n,Lsol_Lmeg[idx_sol_meg,:]), axis=0)
  idx_sol = np.random.choice(Lsol.shape[0], 36, replace=False)
  n = np.concatenate((n,Lsol[idx_sol,:]), axis=0)
  res.append(np.array(n))

traits = np.array(res)

c_traits = traits[:,:,0:28]
d_traits = traits[:,:,28:32]

for i in range(c_traits.shape[2]):
    c_traits[:, :, i] = scalers_BM[i].transform(c_traits[:, :, i]) 

emp_traits_BM = np.concatenate((c_traits, d_traits), axis=2)

c_traits = traits[:,:,0:28]

for i in range(c_traits.shape[2]):
    c_traits[:, :, i] = scalers_OU[i].transform(c_traits[:, :, i]) 

emp_traits_OU = np.concatenate((c_traits, d_traits), axis=2)

In [48]:
Emp_Comb_BM_pred = model1.predict([emp_traits_BM.reshape(100,-1),EmpSNPs])

np.savetxt("Pred_Emp_Comb_BM_Predictions.txt", Emp_Comb_BM_pred)

Emp_SNP_pred = model2.predict(EmpSNPs)

np.savetxt("Pred_Emp_SNP_Predictions.txt", Emp_SNP_pred)

Emp_traits_BM_pred = model3.predict(emp_traits_BM.reshape(100,-1))

np.savetxt("Pred_Emp_traits_BM_Predictions.txt", Emp_traits_BM_pred)

Emp_Comb_OU_pred = model4.predict([emp_traits_OU.reshape(100,-1),EmpSNPs])

np.savetxt("Pred_Emp_Comb_OU_Predictions.txt", Emp_Comb_OU_pred)

Emp_traits_OU_pred = model5.predict(emp_traits_OU.reshape(100,-1))

np.savetxt("Pred_Emp_traits_OU_Predictions.txt", Emp_traits_OU_pred)

4/4 [==============================] - 0s 4ms/step
